In [2]:
from numpy.random import choice
import numpy as np

In [17]:

state_names = ["C1", "C2", "C3", "FB", "Sleep"]

# Probabilities changed to reflect uniform random policy

# Notice Class 3 probabilities reflect possible pub choice:
# Row 3 Column 1:
# (.5 * .2) = .1 = probability of picking pub action (.5) AND
# probability of being sent to class 1 (.2) as a result

# Together they mean a .1 probability 
# of ending up back in C1 from C3

p_matrix = [[0, .5, 0, .5, 0],
            [0, 0, .5, 0, .5],
            [.1, .2, .2, 0, .5],
            [.5, 0, 0, .5, 0],
            [0, 0, 0, 0, 0]]

# Action rewards are weighted and summed by probability of occurring
# I.E: 5.5 = (.5 * 10) + (.5 * 1)
_rewards = [-1.5, -1, 5.5, -.5, 0]


gamma = 1
R = np.array(_rewards)
P = np.matrix(p_matrix)
I = np.identity(len(p_matrix))

solution = np.dot(np.linalg.inv((I-gamma*P)), R)
solution = solution.tolist()[0]
for state in range(len(state_names)):
    print(state_names[state], solution[state])
    

C1 -1.3076923076923084
C2 2.6923076923076925
C3 7.384615384615385
FB -2.3076923076923075
Sleep 0.0


2.0